In [ ]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import geopandas as gpd
import cv2
import numpy as np
import rasterio
from rasterio.enums import Resampling
from skimage import exposure
import matplotlib.pyplot as plt

In [ ]:
api = SentinelAPI('your_username', 'your_password', 'https://scihub.copernicus.eu/dhus')
footprint = geojson_to_wkt(read_geojson('aoi.geojson'))
products = api.query(footprint,
                     date=('20220101', '20220131'),
                     platformname='Sentinel-2',
                     cloudcoverpercentage=(0, 30))
api.download_all(products)

In [ ]:
img_path = 'sentinel_image.tif'
with rasterio.open(img_path) as src:
    img = src.read([1, 2, 3, 4])
    img = np.moveaxis(img, 0, -1)

In [ ]:
def add_radiometric_error(image, noise_level=0.05):
    noise = np.random.normal(scale=noise_level, size=image.shape)
    return np.clip(image + noise, 0, 255)

img_with_radiometric_error = add_radiometric_error(img)

In [ ]:
def apply_geometric_distortion(image, angle=15):
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    rotation_matrix = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (w, h), flags=cv2.INTER_LINEAR)
    return rotated_image

img_with_geometric_distortion = apply_geometric_distortion(img_with_radiometric_error)

In [ ]:
def apply_spectral_error(image, band=0):
    image[:, :, band] = image[:, :, band] * 1.2
    return np.clip(image, 0, 255)
img_with_spectral_error = apply_spectral_error(img_with_geometric_distortion)

In [ ]:
fig, axes = plt.subplots(1, 4, figsize=(20, 5))
axes[0].imshow(img)
axes[0].set_title("Original Image")
axes[1].imshow(img_with_radiometric_error)
axes[1].set_title("Radiometric Error")
axes[2].imshow(img_with_geometric_distortion)
axes[2].set_title("Geometric Distortion")
axes[3].imshow(img_with_spectral_error)
axes[3].set_title("Spectral Error")
plt.show()